## Create sets of SNPs

QC the genotypes

In [ ]:
target=bcac_onco_afr
bash qc_target_data.sh --target ${target} --imp original

Clumping

In [ ]:
discovery=bcac_onco_eur-5pcs
imp=original
target_ref=bcac_onco_eur
target_minor1=bcac_onco_afr
target_minor2=bcac_onco_eas

In [ ]:
pheno=chol
discovery=UKB_${pheno}_gbr
imp=imputeX_full
target_ref=ukbb_gbr
target_minor1=ukbb_afr
target_minor2=ukbb_sas
target_minor3=ukbb_irs
target_minor4=ukbb_white

In [ ]:
cd $PRS_DATASETS/$target_ref/$imp
cat ds.QC.bim | grep rs > ds_rs_snps

In [3]:
pheno=ht
discovery=UKB_${pheno}_gbr
target=ukbb_gbr # $target_ref
imp=imputeX_full

cd $PRS_DATASETS/$target/$imp
cat ds.QC.bim | grep rs > ds_rs_snps

memory=400000
threads=40


plink \
      --bfile ds.QC \
      --extract ds_rs_snps \
      --clump-p1 1 \
      --clump-p2 1 \
      --clump-r2 0.2 \
      --clump-kb 500 \
      --clump $PRS_GWASS/${discovery}/gwas.QC.Transformed \
      --clump-snp-field SNP \
      --clump-field P \
      --out ds.QC_${pheno} \
      --memory ${memory} \
      --threads ${threads} \
      --exclude ds.dupvar


PLINK v1.90b6.24 64-bit (6 Jun 2021)           www.cog-genomics.org/plink/1.9/
(C) 2005-2021 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to ds.QC.log.
Options in effect:
  --bfile ds.QC
  --clump /specific/elkon/hagailevi/PRS/GWASs/bcac_onco_eur-5pcs/gwas.QC.Transformed
  --clump-field P
  --clump-kb 500
  --clump-p1 1
  --clump-p2 1
  --clump-r2 0.2
  --clump-snp-field SNP
  --exclude ds.dupvar
  --memory 400000
  --out ds.QC
  --threads 40

1019915 MB RAM detected; reserving 400000 MB for main workspace.
338487 variants loaded from .bim file.
123906 people (0 males, 123906 females) loaded from .fam.
--exclude: 338487 variants remaining.



### Extract representative SNPs using only major target set

In [7]:
common_prefix="single"

In [ ]:
target_ref="ukbb_gbr"
imp="imputeX_gbr-snps"

cd $PRS_DATASETS
cat $target_ref/$imp/ds.QC_${pheno}.clumped | awk '{print $3}' | grep rs | sort > $target_ref/$imp/${common_prefix}_${pheno}_clumped_snps

### Extract representative SNPs using all target sets

In [5]:
common_prefix="common"

Combine two minor target sets

In [6]:
target_ref=ukbb_gbr
target_minor1=ukbb_afr
target_minor2=ukbb_sas
imp=imputeX_full

cd $PRS_DATASETS
comm -12 <(cat $target_ref/$imp/ds.QC_${pheno}.clumped | awk '{print $3}' | sort) <(comm -12 <(cat  $target_minor1/$imp/ds.QC_${pheno}.clumped | awk '{print $3}' | sort) <(cat  $target_minor2/$imp/ds.QC_${pheno}.clumped | awk '{print $3}' | sort) | sort) | grep rs | > $target_ref/$imp/${common_prefix}_${pheno}_clumped_snps

cat: ukbb_sas/original/ds.QC.clumped: No such file or directory
cat: ukbb_afr/original/ds.QC.clumped: No such file or directory
cat: ukbb_gbr/original/ds.QC.clumped: No such file or directory


Combine four minor target sets

In [ ]:
target_ref=ukbb_gbr
target_minor1=ukbb_afr
target_minor2=ukbb_sas
target_minor3=ukbb_irs
target_minor4=ukbb_white
imp=imputeX_full
pheno=t2d

comm -12 <(cat $target_ref/$imp/ds.QC_${pheno}.clumped | awk '{print $3}' | sort) <(comm -12 <(cat  $target_minor1/$imp/ds.QC_${pheno}.clumped | awk '{print $3}' | sort) <(comm -12 <(cat  $target_minor2/$imp/ds.QC_${pheno}.clumped | awk '{print $3}' | sort) <(comm -12 <(cat  $target_minor3/$imp/ds.QC_${pheno}.clumped | awk '{print $3}' | sort) <(cat  $target_minor4/$imp/ds.QC_${pheno}.clumped | awk '{print $3}' | sort)))) | grep rs > $target_ref/$imp/${common_prefix}_${pheno}_clumped_snps

### Downstream analysis

Optional: check SNPs abundance in each chromosome

In [ ]:
cd $PRS_DATASETS
awk '{if(NR==FNR){c[$1]++; next;}; if(c[$2]>0){print $1" "$2}}' $target_ref/$imp/${common_prefix}_${pheno}_clumped_snps  $target_ref/$imp/ds.QC.bim | awk '{print $1}' | sort | uniq -cd | sort

Extract details of SNPs, including pvals

In [ ]:
target_ref=ukbb_gbr
cd $PRS_DATASETS
awk '{if(NR==FNR){c[$1]++; next;}if(c[$3]>0){print $0} }' $target_ref/$imp/${common_prefix}_${pheno}_clumped_snps $target_ref/$imp/ds.QC_${pheno}.clumped | tr -s ' ' | cut -d ' ' -f 1-6 > $target_ref/$imp/${common_prefix}_${pheno}_clumped_snps_details

Filter SNPs by pvalue threshold

In [ ]:
ths=("0.000000000001" "0.0000000000001" "0.00000000000001") # ("0.1" "0.01" "0.001" "0.0001" "0.00001"  "0.00001" "0.000001" "0.0000001" "0.00000001")

cd $PRS_DATASETS
for th in ${ths[@]}; do
    cat  $target_ref/$imp/${common_prefix}_${pheno}_clumped_snps_details | awk '{if($5<='$th'){print $3}}' > $target_ref/$imp/${common_prefix}_${pheno}_clumped_snps_${th}
done

## Extract SNPs according to the sets generated above

Init params

In [ ]:
targets=("bcac_onco_afr" "bcac_onco_eas" "bcac_onco_eur")
imp=original
ths=("0.1" "0.01" "0.001")

In [1]:
targets=("ukbb_gbr") #  "ukbb_sas" "ukbb_afr" "ukbb_irs" "ukbb_white")
imp="imputeX_full" # "imputeX_gbr-snps"
ths=("0.0000001")
common_prefix="single" # "common"

Refactor bed file to tsv file format

In [ ]:
$target_ref="ukbb_gbr"
targets=("ukbb_gbr" "ukbb_sas" "ukbb_afr" "ukbb_irs" "ukbb_white")
imp="imputeX_full"
ths=("0.001") #  "0.01" "0.001")

for th in ${ths[@]}; do
    for target in ${targets[@]}; do 
        cd $PRS_DATASETS/$target/$imp
        rm *_no_headers.vcf
        plink --bfile ds --extract $PRS_DATASETS/$target_ref/$imp/${common_prefix}_${pheno}_clumped_snps_${th} --recode vcf --out ds.QC_${common_prefix}_${pheno}_${th} && \
        cat ds.QC_${common_prefix}_${pheno}_${th}.vcf | grep -v '##' | awk '{if (FNR==1){print substr($0,2,length($0))} else {print $0}}' > ds.QC_${common_prefix}_${pheno}_${th}_no_comments.vcf
    done
done

In [ ]:
target_ref="ukbb_gbr"
targets=("ukbb_sas" "ukbb_afr" "ukbb_irs" "ukbb_white" "ukbb_gbr")
imp="imputeX_full"
ths=("0.001") #  "0.01" "0.001")

for th in ${ths[@]}; do
    for target in ${targets[@]}; do 
        cd $PRS_DATASETS/$target/$imp
        plink --bfile ds --extract $PRS_DATASETS/$target_ref/$imp/${common_prefix}_${pheno}_clumped_snps_${th} --make-bed --out ds.QC_${common_prefix}_${pheno}_${th}
    done
done

Convert vcf cell format (*/*) to minor allele counts (0-2)

In [4]:
targets=("ukbb_gbr" "ukbb_sas" "ukbb_afr" "ukbb_irs" "ukbb_white")
imp="imputeX_full"
ths=("0.001") # ("0.1" "0.01" "0.001")

for th in ${ths[@]}; do # 0.01 0.001; do # 0.1 0.01 0.001 
    for target in ${targets[@]}; do
        cd $PRS_DATASETS/$target/$imp
        python -c \
'
import pandas as pd
d_enc={"0/0" : "0", "0/1": "1", "1/0" : "1", "1/1" : 2}
df=pd.read_csv("ds.QC_'$common_prefix'_'$pheno'_'$th'_no_comments.vcf", delim_whitespace=True, index_col=2).iloc[:,8:]
df.columns=[col.split("_")[0] for col in df.columns]
df=df.applymap(lambda a: d_enc.get(a, "-1"))
df.to_csv("ds_'$common_prefix'_'$pheno'_'$th'.tsv", sep="\t")
'
    done
done

Traceback (most recent call last):
  File "<string>", line 2, in <module>
  File "/home/gaga/hagailevi/anaconda3/lib/python3.8/site-packages/pandas/__init__.py", line 30, in <module>
    from pandas._libs import hashtable as _hashtable, lib as _lib, tslib as _tslib
  File "/home/gaga/hagailevi/anaconda3/lib/python3.8/site-packages/pandas/_libs/__init__.py", line 13, in <module>
    from pandas._libs.interval import Interval
  File "pandas/_libs/interval.pyx", line 1, in init pandas._libs.interval
  File "pandas/_libs/hashtable.pyx", line 1, in init pandas._libs.hashtable
  File "pandas/_libs/missing.pyx", line 1, in init pandas._libs.missing
  File "/home/gaga/hagailevi/anaconda3/lib/python3.8/site-packages/pandas/_libs/tslibs/__init__.py", line 30, in <module>
    from .conversion import OutOfBoundsTimedelta, localize_pydatetime
  File "pandas/_libs/tslibs/conversion.pyx", line 58, in init pandas._libs.tslibs.conversion
  File "pandas/_libs/tslibs/parsing.pyx", line 34, in init pandas

In [ ]:
targets=("ukbb_gbr") # ("ukbb_gbr" "ukbb_sas" "ukbb_afr" "ukbb_irs" "ukbb_white")
imp="imputeX_full"
ths=("0.01") # ("0.1" "0.01" "0.001")

for th in ${ths[@]}; do 
    for target in ${targets[@]}; do
        cd $PRS_DATASETS/$target/$imp
        # cut -f 1 ds_${th}.tsv > ds_${th}_index.tsv
        cat ds.QC_${common_prefix}_${pheno}_${th}_no_comments.vcf | cut -f 3 | tail -n +2 > ds_${common_prefix}_${pheno}_${th}_index.tsv
        head -n 1 ds_${common_prefix}_${pheno}_${th}.tsv | cut -f 2- > ds_${common_prefix}_${pheno}_${th}_header.tsv
        tail -n +2 ds_${common_prefix}_${pheno}_${th}.tsv | cut -f 2- > ds_${common_prefix}_${pheno}_${th}_data.tsv
    done
done

In [ ]:
for th in ${ths[@]}; do 
    for target in ${targets[@]}; do
        cd $PRS_DATASETS/$target/$imp
        python -c \
'
import pandas as pd
df=pd.read_csv("ds.QC_'${common_prefix}'_'${pheno}'_'${th}'.bim", delim_whitespace=True, header=None)
for chr,sub_df in df.groupby(0):
    print(f"{chr}\t{'"','"'.join(list(sub_df[1]))}")
' > full_chrs_mutual_${common_prefix}_${pheno}_${th}
    done
done

### generate datasets by a predefined set of SNPs

In [ ]:
target_ref="ukbb_gbr"
targets=("ukbb_sas" "ukbb_afr" "ukbb_irs" "ukbb_white" "ukbb_gbr")
imp="imputeX_full"
ds_prefix="union_all"
pheno="t2d"

In [ ]:
cd $PRS_DATASETS
for target in ${targets[@]}; do 
    plink --bfile $target/$imp/ds.QC --extract $target_ref/${imp}/clumped_snps_${ds_prefix}_${pheno} --make-bed --out $target/$imp/ds.QC_${ds_prefix}_${pheno}; 
done

In [ ]:
for target in ${targets[@]}; do 
    cd $PRS_DATASETS/$target/$imp
    rm *_no_headers.vcf
    plink --bfile ds.QC_${ds_prefix}_${pheno} --recode vcf --out ds.QC_${ds_prefix}_${pheno} && \
    cat ds.QC_${ds_prefix}_${pheno}.vcf | grep -v '##' | awk '{if (FNR==1){print substr($0,2,length($0))} else {print $0}}' > ds.QC_${ds_prefix}_${pheno}_no_comments.vcf
done


Convert vcf cell format (*/*) to minor allele counts (0-2)

In [4]:
for target in ${targets[@]}; do
    cd $PRS_DATASETS/$target/$imp
    echo "start converting $target"
    python -c \
'
import pandas as pd
d_enc={"0/0" : "0", "0/1": "1", "1/0" : "1", "1/1" : 2}
df=pd.read_csv("ds.QC_'$ds_prefix'_'$pheno'_no_comments.vcf", delim_whitespace=True, index_col=2).iloc[:,8:]
df.columns=[col.split("_")[0] for col in df.columns]
df=df.applymap(lambda a: d_enc.get(a, "-1"))
df.to_csv("ds_'$ds_prefix'_'$pheno'.tsv", sep="\t")
'
done

Traceback (most recent call last):
  File "<string>", line 2, in <module>
  File "/home/gaga/hagailevi/anaconda3/lib/python3.8/site-packages/pandas/__init__.py", line 30, in <module>
    from pandas._libs import hashtable as _hashtable, lib as _lib, tslib as _tslib
  File "/home/gaga/hagailevi/anaconda3/lib/python3.8/site-packages/pandas/_libs/__init__.py", line 13, in <module>
    from pandas._libs.interval import Interval
  File "pandas/_libs/interval.pyx", line 1, in init pandas._libs.interval
  File "pandas/_libs/hashtable.pyx", line 1, in init pandas._libs.hashtable
  File "pandas/_libs/missing.pyx", line 1, in init pandas._libs.missing
  File "/home/gaga/hagailevi/anaconda3/lib/python3.8/site-packages/pandas/_libs/tslibs/__init__.py", line 30, in <module>
    from .conversion import OutOfBoundsTimedelta, localize_pydatetime
  File "pandas/_libs/tslibs/conversion.pyx", line 58, in init pandas._libs.tslibs.conversion
  File "pandas/_libs/tslibs/parsing.pyx", line 34, in init pandas

In [ ]:
for target in ${targets[@]}; do
    cd $PRS_DATASETS/$target/$imp
    # cut -f 1 ds_${th}.tsv > ds_${th}_index.tsv
    cat ds.QC_${ds_prefix}_${pheno}_no_comments.vcf | cut -f 3 | tail -n +2 > ds_${ds_prefix}_${pheno}_index.tsv
    head -n 1 ds_${ds_prefix}_${pheno}_.tsv | cut -f 2- > ds_${ds_prefix}_${pheno}_header.tsv
    tail -n +2 ds_${ds_prefix}_${pheno}_.tsv | cut -f 2- > ds_${ds_prefix}_${pheno}_data.tsv
done

In [ ]:
for target in ${targets[@]}; do
    cd $PRS_DATASETS/$target/$imp
    python -c \
'
import pandas as pd
df=pd.read_csv("ds.QC_'${ds_prefix}'_'${pheno}'.bim", delim_whitespace=True, header=None)
for chr,sub_df in df.groupby(0):
    print(f"{chr}\t{'"','"'.join(list(sub_df[1]))}")
' > full_chrs_mutual_${ds_prefix}_${pheno}
done
